**1.2 Spreading on complex network (Modelling and Machine Learning)**

Jupyter Notebook (.ipynb) Answers to Questions 4a, 4b and 5a

Commonwealth Secondary School | Aarav Malik, Michelle Aurelia Yudianto, Pua Hong Wei 

Importing of modules and reading of _.edgelist_ file:

In [29]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

G = nx.read_edgelist("contact.edgelist")


# nx.draw(G, with_labels=True)
# plt.show()

**4a)**

Function to calculate number of infections 

In [30]:
def infections(third):
    if third in G.nodes():
        infected = ['8', '9']
        infected += [third]
        for i in range(36):
            for node in G.nodes():
                if node not in infected:
                    if len(list(set(G.neighbors(node)).intersection(infected))) >= 2:
                        infected += [node]

        return infected
    else:
        return 'X'

Function to calculate infections post-vaccination

In [31]:
def vaccine(target):
    if target in G.nodes():
        G.remove_node(target)
        infected = ['8', '9', '151']
        for i in range(36):
            for node in G.nodes():
                if node not in infected:
                    if len(list(set(G.neighbors(node)).intersection(infected))) >= 2:
                        infected += [node]
        return infected
    else:
        return 'X'

Finding the highest number of infected people possible:

In [32]:
maxer = []
for person in range(152):
    maxer += [len(infections(str(person)))]
print("Highest number of infected people possible: ", max(maxer))  # node 2 and 151

Highest number of infected people possible:  22


**4b)**

Finding number of infections after vaccination for each node:

In [33]:
miner = []
for human in range(152):
    G = nx.read_edgelist("contact.edgelist")
    if str(human) in G.nodes():
        miner += [len(vaccine(str(human)))]
    else:
        miner += [1]
    # nx.draw(G, with_labels=True)
    # plt.show()

Finding the node which would result in the largest decrease of infections

In [34]:
G = nx.read_edgelist("contact.edgelist")
diff = []
ndiff = []
for num in miner:
    if num != 1:
        diff += [len(infections('151')) - num]
        ndiff += [(len(infections('151')) - num) / 36 * 100]
    else:
        diff += [1]
diff[151], diff[1], diff[8], diff[9] = 0, 0, 0, 0
ndiff[-1], ndiff[0], ndiff[3], ndiff[4] = 0, 0, 0, 0
print("Recommended node to be vaccinated: ", diff.index(max(diff)))

Recommended node to be vaccinated:  91


**5a)**

Data organisation: 

In [35]:
nd = []
for n in G.nodes():
    nd += [int(n)]
nd = sorted(nd)
plens = []

for z in range(152):
    if str(z) in G.nodes():
        plen = nx.shortest_path_length(G, '151', str(z))
        plens += [plen]

y_train = np.array(plens).reshape(-1, 1)
x_train = np.array(ndiff).reshape(-1, 1)

Training of linear regression model:

In [36]:
model = LinearRegression()
model.fit(x_train, y_train)
y = model.predict(x_train)
print("Model Score: ", model.score(x_train, y_train))
print("Mean Square error:", mean_squared_error(y_train, y))
# plt.plot(ndiff, plens, '.')
# plt.show()

Model Score:  0.40324554700454207
Mean Square error: 0.34764630556448356


Calculating vaccination effectiveness of input node:

In [37]:
entity = int(input("To be vaccinated: "))
pos = nd.index(entity)
contacts = plens[pos]
x = (contacts - model.intercept_)/model.coef_
print("Effectiveness: ", float(x), "%")

Effectiveness:  40.06688241639699 %
